<a href="https://colab.research.google.com/github/tiwaripari/ML/blob/main/Genreclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import librosa
import librosa.display
import IPython.display as ipd
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier

In [2]:
df = pd.read_csv("/content/drive/MyDrive/features_3_sec.csv")

In [3]:
df

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.0.wav,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,blues.00000.1.wav,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,blues.00000.2.wav,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,blues.00000.3.wav,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,blues.00000.4.wav,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9985,rock.00099.5.wav,66149,0.349126,0.080515,0.050019,0.000097,1499.083005,164266.886443,1718.707215,85931.574523,...,42.485981,-9.094270,38.326839,-4.246976,31.049839,-5.625813,48.804092,1.818823,38.966969,rock
9986,rock.00099.6.wav,66149,0.372564,0.082626,0.057897,0.000088,1847.965128,281054.935973,1906.468492,99727.037054,...,32.415203,-12.375726,66.418587,-3.081278,54.414265,-11.960546,63.452255,0.428857,18.697033,rock
9987,rock.00099.7.wav,66149,0.347481,0.089019,0.052403,0.000701,1346.157659,662956.246325,1561.859087,138762.841945,...,78.228149,-2.524483,21.778994,4.809936,25.980829,1.775686,48.582378,-0.299545,41.586990,rock
9988,rock.00099.8.wav,66149,0.387527,0.084815,0.066430,0.000320,2084.515327,203891.039161,2018.366254,22860.992562,...,28.323744,-5.363541,17.209942,6.462601,21.442928,2.354765,24.843613,0.675824,12.787750,rock


In [4]:
df.label.value_counts()

blues        1000
jazz         1000
metal        1000
pop          1000
reggae       1000
disco         999
classical     998
hiphop        998
rock          998
country       997
Name: label, dtype: int64

In [5]:
features = df.columns[2:59]
features

Index(['chroma_stft_mean', 'chroma_stft_var', 'rms_mean', 'rms_var',
       'spectral_centroid_mean', 'spectral_centroid_var',
       'spectral_bandwidth_mean', 'spectral_bandwidth_var', 'rolloff_mean',
       'rolloff_var', 'zero_crossing_rate_mean', 'zero_crossing_rate_var',
       'harmony_mean', 'harmony_var', 'perceptr_mean', 'perceptr_var', 'tempo',
       'mfcc1_mean', 'mfcc1_var', 'mfcc2_mean', 'mfcc2_var', 'mfcc3_mean',
       'mfcc3_var', 'mfcc4_mean', 'mfcc4_var', 'mfcc5_mean', 'mfcc5_var',
       'mfcc6_mean', 'mfcc6_var', 'mfcc7_mean', 'mfcc7_var', 'mfcc8_mean',
       'mfcc8_var', 'mfcc9_mean', 'mfcc9_var', 'mfcc10_mean', 'mfcc10_var',
       'mfcc11_mean', 'mfcc11_var', 'mfcc12_mean', 'mfcc12_var', 'mfcc13_mean',
       'mfcc13_var', 'mfcc14_mean', 'mfcc14_var', 'mfcc15_mean', 'mfcc15_var',
       'mfcc16_mean', 'mfcc16_var', 'mfcc17_mean', 'mfcc17_var', 'mfcc18_mean',
       'mfcc18_var', 'mfcc19_mean', 'mfcc19_var', 'mfcc20_mean', 'mfcc20_var'],
      dtype='object')

In [6]:
#Creating Test and Train data
df['is_train'] = np.random.uniform(0, 1, len(df)) <= .75
#View top 10 rows
df.head(10)

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label,is_train
0,blues.00000.0.wav,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,...,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues,False
1,blues.00000.1.wav,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,...,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues,True
2,blues.00000.2.wav,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,...,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues,True
3,blues.00000.3.wav,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,...,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues,False
4,blues.00000.4.wav,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,...,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues,True
5,blues.00000.5.wav,66149,0.376670,0.089702,0.132618,0.003583,1994.915219,211700.619569,2152.767854,74263.873102,...,-3.448373,34.284130,-0.416165,40.791092,-3.649625,32.457901,3.025218,28.892687,blues,True
6,blues.00000.6.wav,66149,0.379909,0.088827,0.130335,0.003166,1962.150096,177443.070045,2146.503479,98020.541422,...,-2.068194,25.623655,1.428141,47.957699,-3.267124,39.382240,3.276939,25.999132,blues,True
7,blues.00000.7.wav,66149,0.331880,0.092119,0.140600,0.002546,1701.890924,35678.130616,1979.387612,36670.725886,...,-3.637886,24.530296,-0.105148,26.716150,-2.016985,23.150423,0.210787,42.512966,blues,True
8,blues.00000.8.wav,66149,0.347877,0.094209,0.133130,0.002538,1746.473502,138073.931244,1887.619723,117069.920049,...,-4.370029,29.873167,2.114592,33.843155,-2.264663,80.812393,3.758598,97.618835,blues,True
9,blues.00000.9.wav,66149,0.358061,0.082957,0.115312,0.001846,1763.948942,61493.423121,1874.195710,51944.921435,...,-3.448304,48.671944,0.099792,41.839546,-7.677177,96.253654,0.791776,40.416420,blues,True


In [7]:
#Create dataframes with train and test rows
train, test = df[df['is_train']==True], df[df['is_train']==False]

#show the number of rows in test and train dataframe
print("training data ", len(train))
print("testing data ", len(test))

training data  7417
testing data  2573


In [8]:
y = pd.factorize(train['label'])[0]
y

array([0, 0, 0, ..., 9, 9, 9])

In [9]:
x = pd.factorize(test['label'])[0]
x

array([0, 0, 0, ..., 9, 9, 9])

In [10]:
target_names = (train['label']).unique()
target_names

array(['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz',
       'metal', 'pop', 'reggae', 'rock'], dtype=object)

In [20]:
#Creating Random Forest Classifier
#clf = RandomForestClassifier(n_jobs=2, random_state=0)
clf = XGBClassifier()
clf.fit(train[features], y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [21]:
#Applying trained Classifier to the test
predictions = clf.predict(test[features])
predictions

array([0, 2, 8, ..., 1, 9, 9])

In [22]:
#viewing predicted probablities of first 10 observation
clf.predict_proba(test[features])[:10]

array([[8.2515824e-01, 1.7208871e-05, 1.3038275e-01, 5.8356915e-03,
        6.5190463e-05, 7.1762668e-05, 2.8782739e-05, 2.2438669e-05,
        1.2587368e-03, 3.7159245e-02],
       [2.4439499e-01, 4.0326831e-05, 7.1957171e-01, 8.9893332e-03,
        3.4454756e-04, 5.3269461e-05, 1.2221570e-04, 5.3252796e-05,
        5.7438749e-04, 2.5855957e-02],
       [6.9819957e-02, 1.3644794e-04, 1.5371300e-02, 1.2439010e-02,
        3.2101667e-03, 2.8595058e-04, 2.6856964e-03, 1.0116422e-03,
        8.5485870e-01, 4.0181089e-02],
       [9.7713542e-01, 5.9138652e-05, 3.0034606e-04, 3.0781359e-03,
        8.3310594e-04, 2.6011706e-04, 3.0060122e-05, 2.3951271e-04,
        1.4872860e-02, 3.1912681e-03],
       [9.0303022e-01, 1.5946278e-04, 7.1332343e-02, 1.2444998e-03,
        2.9509314e-04, 1.0253596e-04, 1.6481002e-04, 5.1509956e-05,
        1.5559795e-03, 2.2063537e-02],
       [9.7007555e-01, 2.5160282e-04, 2.7550829e-03, 9.7303302e-05,
        8.0627426e-05, 4.1125513e-05, 3.9409584e-05, 1.09

In [23]:
#mapping names of moods for each music for each predicted mood
preds  = target_names[clf.predict(test[features])]

#View the predicted moods for firat five observations
preds[10:20]

array(['blues', 'blues', 'blues', 'blues', 'blues', 'country', 'blues',
       'country', 'blues', 'blues'], dtype=object)

In [24]:
#Creating confusion matrix
#crosstab takes two sets of data and forms chart out of it
pd.crosstab(test['label'], preds, rownames=['Actual Label'], colnames=['Predicted Label'])

Predicted Label,blues,classical,country,disco,hiphop,jazz,metal,pop,reggae,rock
Actual Label,,,,,,,,,,
blues,236,0,10,1,2,2,1,0,11,3
classical,0,236,2,0,0,11,0,0,0,2
country,11,1,215,0,1,4,0,2,3,4
disco,0,2,5,233,3,6,0,4,4,6
hiphop,3,1,4,4,213,2,1,4,5,5
jazz,1,10,9,1,0,232,0,0,2,1
metal,5,0,1,0,4,1,238,0,1,5
pop,0,0,3,3,4,2,0,240,2,5
reggae,2,2,5,3,8,4,1,9,233,3


In [25]:
accuracy = accuracy_score(x, predictions) 
accuracy*100

89.31208705790905